In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
module_path = os.path.abspath(os.path.join('Mask2Former'))
if module_path not in sys.path:
    sys.path.append(module_path)
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import polars as pl
import sklearn as sklearn
import scipy as scipy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sys
import torch as pytorch
import cv2 as opencv
print(f"Polars Version: {pl.__version__}")
print(f"scikit-learn Version: {sklearn.__version__}")
print(f"scipy Version: {scipy.__version__}")
print(f"pandas Version: {pd.__version__}")
print(f"matplotlib Version: {matplotlib.__version__}")
print(f"PyTorch Version: {pytorch.__version__}")
print(f"OpenCV Version: {opencv.__version__}")
gpu = pytorch.cuda.is_available()
print("GPU is", "available" if gpu else "NOT AVAILABLE")

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
setup_logger(name="mask2former")
import numpy as np
import cv2
import torch
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2.projects.deeplab import add_deeplab_config
coco_metadata = MetadataCatalog.get("coco_2017_val_panoptic")
from mask2former import add_maskformer2_config

In [ ]:
im = cv2.imread("data/smartphone_1.jpg")
plt.figure(figsize = (20,20))
plt.imshow(im[:, :, ::-1])
plt.show()

In [ ]:
cfg = get_cfg()
add_deeplab_config(cfg)
add_maskformer2_config(cfg)
cfg.merge_from_file("Mask2Former/configs/coco/panoptic-segmentation/swin/maskformer2_swin_large_IN21k_384_bs16_100ep.yaml")
cfg.MODEL.WEIGHTS = 'https://dl.fbaipublicfiles.com/maskformer/mask2former/coco/panoptic/maskformer2_swin_large_IN21k_384_bs16_100ep/model_final_f07440.pkl'
cfg.MODEL.MASK_FORMER.TEST.SEMANTIC_ON = False
cfg.MODEL.MASK_FORMER.TEST.INSTANCE_ON = True
cfg.MODEL.MASK_FORMER.TEST.PANOPTIC_ON = True
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [ ]:
outputs = predictor(im)

In [ ]:
v = Visualizer(im[:, :, ::-1], coco_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
instance_result = v.draw_instance_predictions(outputs["instances"].to("cpu")).get_image()
v = Visualizer(im[:, :, ::-1], coco_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
plt.figure(figsize = (20,20))
plt.imshow(instance_result[:, :, ::-1])
plt.show()

In [ ]:
img_height, img_width = im.shape[0], im.shape[1]
masks = outputs["instances"].pred_masks.to("cpu")
pred_classes = outputs["instances"].pred_classes.to("cpu")
n_channels = 4
transparent_img = np.zeros((img_height, img_width, n_channels), dtype=np.uint8)
phone_label_index = np.argmax(outputs["instances"].scores.cpu().numpy())
if (pred_classes[phone_label_index]==67):
    for h in range(img_height):
        for w in range(img_width):
            #if(mask_1[h][w]==1):
            if(masks[phone_label_index][h][w]==1):
                transparent_img[h][w] = [im[h][w][0], im[h][w][1], im[h][w][2], 255]
plt.figure(figsize = (20,20))
plt.imshow(transparent_img)
plt.show()